In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

## Data Preparation

In [1]:
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import RandomizedSearchCV, train_test_split;
import numpy;
import pandas;
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier;
import sys;
from keras.utils import to_categorical;
REMOTE_MODULE_PATH = '/home/macbookretina/automatic-music-genre-classification/feature_extraction_deep_learning'
# LOCAL_MODULE_PATH = '/Users/macbookretina/Desktop/automatic-music-genre-classification/feature_extraction_deep_learning'
sys.path.insert(1, REMOTE_MODULE_PATH)
from custom_module.utilities import *

Using TensorFlow backend.


In [2]:
features = pandas.read_csv(MOUNTED_DATASET_PATH + '/data/cleaned_features.csv', index_col=0)
features

,genre_label,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,0,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,0,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,0,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,0,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,0,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,0,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,2,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,2,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [4]:
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]
X

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,mean_db4_cD3,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,-0.000069,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,0.000984,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,0.000019,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,-0.001993,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,0.000076,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,-0.000065,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,-0.000009,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,-0.000024,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,-0.000132,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [5]:
y.value_counts(normalize=True)

2    0.333434
0    0.333434
1    0.333131
Name: genre_label, dtype: float64

In [6]:
wavelet_predictors = X.filter(regex=(r'.+_db[458]{1}_.+'))
wavelet_predictor_labels = wavelet_predictors.columns.values
timbral_rhythmic_predictors = X.loc[:, features.columns.difference(
    numpy.append(wavelet_predictor_labels, 'genre_label'))]
timbral_rhythmic_predictors

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_mfcc_1,mean_mfcc_10,mean_mfcc_11,mean_mfcc_12,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-80.892197,19.216515,-13.114045,11.498654,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-81.181618,20.319712,-2.532140,12.753151,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-202.881241,9.440219,-11.644846,7.070107,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-40.265057,16.794779,-3.511151,8.467217,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-140.261337,16.041454,-3.190496,11.844710,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,-285.722076,1.724312,3.334758,-1.919123,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,-325.716064,-5.354380,-14.410619,-4.959855,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,-264.064270,0.347791,5.412341,5.308122,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-178.481781,4.596525,6.592151,10.097037,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [7]:
predictors_all_label = timbral_rhythmic_predictors.columns.values
predictors_with_outliers_label = ['mean_beats', 'mean_spec_bw', 'mean_spec_centroid', 'mean_spec_rolloff',
                          'median_beats','median_spec_bw', 'median_spec_centroid', 
                          'median_spec_rolloff', 'std_beats', 'std_mfcc_1', 'var_spec_centroid',
                          'var_spec_rolloff'
                         ]
predictors_without_outliers_label = timbral_rhythmic_predictors.loc[
    :, timbral_rhythmic_predictors.columns.difference(predictors_with_outliers_label)].columns.values

In [8]:
preprocess_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label)
preprocess_np = normalization_pipeline(
    predictors_all_label)

### Split & Transform Data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    timbral_rhythmic_predictors, y, test_size=0.2, shuffle=True)

In [10]:
pipeline_estimator_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label
).fit(X_train)
# X_train_sp = pipeline_estimator_sp.transform(X_train)
X_test_sp = pipeline_estimator_sp.transform(X_test)
X_test_sp.shape

(660, 109, 1)

In [11]:
pipeline_estimator_np = normalization_pipeline(
    predictors_all_label).fit(X_train)
# X_train_np = pipeline_estimator_np.transform(X_train)
X_test_np = pipeline_estimator_np.transform(X_test)
X_test_np.shape

(660, 109, 1)

In [12]:
# X_train_sp = standardization_pipeline(wavelet_predictor_labels, 
#                                    [], wavelet_predictor_labels
# #                                   ).fit_transform(X_train)
# X_test_sp = standardization_pipeline(
#     predictors_all_label,
#     predictors_with_outliers_label,
#     predictors_without_outliers_label
# ).fit_transform(X_test)
# X_test_sp.shape

In [13]:
# X_train_np = normalization_pipeline(wavelet_predictor_labels
# #                                   ).fit_transform(X_train)
# X_test_np = normalization_pipeline(predictors_all_label
#                                   ).fit_transform(X_test)
# X_test_np.shape

In [14]:
_, ncols = X_train.shape

In [15]:
# one hot encoding our labels
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [16]:
y_train.shape, y_test.shape

((2639, 3), (660, 3))

## Predictive Modelling

In [17]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [18]:
param_dist = {
    'clf__n_hidden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100)
}

### FNN With Standardized Input

In [19]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=set_shape_create_model('fnn', ncols))

In [20]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', fnn_1)
])

In [21]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [22]:
# %%timeit
# fit model
search_1 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 109)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 330       
Total params: 330
Trainable params: 330
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 5ms/step - loss: 1.5344 - accuracy: 0.4161 - val_loss: 1.3924 - val_accuracy: 0.4606
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.3787 - accuracy: 0.4525 - val_loss: 1.2956 - val_accuracy: 0.4712
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 1.2925 - accuracy: 0.4707 - val_loss: 1.2303 - val_accuracy: 0.4894
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 1.2305 - accuracy: 0.4827 

55/55 [==============================] - 0s 2ms/step - loss: 1.1136 - accuracy: 0.5304 - val_loss: 1.1173 - val_accuracy: 0.5227
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0842 - accuracy: 0.5372 - val_loss: 1.0911 - val_accuracy: 0.5333
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0595 - accuracy: 0.5446 - val_loss: 1.0690 - val_accuracy: 0.5409
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0386 - accuracy: 0.5509 - val_loss: 1.0501 - val_accuracy: 0.5470
Epoch 10/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0208 - accuracy: 0.5560 - val_loss: 1.0338 - val_accuracy: 0.5515
Epoch 11/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0053 - accuracy: 0.5600 - val_loss: 1.0196 - val_accuracy: 0.5561
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9918 - accuracy: 0.5696 - val_loss: 1.0072 - val_accuracy: 0.5636
Epoch 13/100
55/55 [==

Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8324 - accuracy: 0.6453 - val_loss: 0.8518 - val_accuracy: 0.6394
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8315 - accuracy: 0.6464 - val_loss: 0.8508 - val_accuracy: 0.6424
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8306 - accuracy: 0.6464 - val_loss: 0.8499 - val_accuracy: 0.6424
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8296 - accuracy: 0.6470 - val_loss: 0.8490 - val_accuracy: 0.6424
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8288 - accuracy: 0.6475 - val_loss: 0.8481 - val_accuracy: 0.6409
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8279 - accuracy: 0.6470 - val_loss: 0.8472 - val_accuracy: 0.6394
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8270 - accuracy: 0.6470 - val_loss: 0.8464 - val_accuracy: 0.6394
Epoch 

Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8587 - accuracy: 0.6347 - val_loss: 0.8544 - val_accuracy: 0.6439
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8579 - accuracy: 0.6358 - val_loss: 0.8536 - val_accuracy: 0.6439
Epoch 76/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8572 - accuracy: 0.6358 - val_loss: 0.8528 - val_accuracy: 0.6424
Epoch 77/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8564 - accuracy: 0.6358 - val_loss: 0.8520 - val_accuracy: 0.6439
Epoch 78/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8557 - accuracy: 0.6364 - val_loss: 0.8512 - val_accuracy: 0.6439
Epoch 79/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8549 - accuracy: 0.6352 - val_loss: 0.8505 - val_accuracy: 0.6439
Epoch 80/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8542 - accuracy: 0.6364 - val_loss: 0.8497 - val_accuracy: 0.6439
Epoch 

Epoch 1/100
55/55 [==============================] - 0s 4ms/step - loss: 1.0308 - accuracy: 0.5099 - val_loss: 0.8848 - val_accuracy: 0.5606
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8412 - accuracy: 0.5918 - val_loss: 0.8152 - val_accuracy: 0.6091
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7803 - accuracy: 0.6384 - val_loss: 0.7726 - val_accuracy: 0.6515
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7337 - accuracy: 0.6708 - val_loss: 0.7480 - val_accuracy: 0.6667
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7096 - accuracy: 0.6890 - val_loss: 0.7210 - val_accuracy: 0.6818
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6890 - accuracy: 0.7015 - val_loss: 0.7059 - val_accuracy: 0.7076
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6742 - accuracy: 0.7118 - val_loss: 0.6991 - val_accuracy: 0.7061
Epoch 8/100
5

...

55/55 [==============================] - 0s 2ms/step - loss: 0.6837 - accuracy: 0.7074 - val_loss: 0.6974 - val_accuracy: 0.7136
Epoch 30/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6834 - accuracy: 0.7057 - val_loss: 0.6944 - val_accuracy: 0.7121
Epoch 31/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6807 - accuracy: 0.7108 - val_loss: 0.6896 - val_accuracy: 0.7212
Epoch 32/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6824 - accuracy: 0.7063 - val_loss: 0.6904 - val_accuracy: 0.7182
Epoch 33/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6788 - accuracy: 0.7068 - val_loss: 0.6885 - val_accuracy: 0.7212
Epoch 34/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6770 - accuracy: 0.7080 - val_loss: 0.6947 - val_accuracy: 0.7076
Epoch 35/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6757 - accuracy: 0.7108 - val_loss: 0.6936 - val_accuracy: 0.7121
Epoch 36/100
55/55 

55/55 [==============================] - 0s 3ms/step - loss: 0.5265 - accuracy: 0.7784 - val_loss: 0.7640 - val_accuracy: 0.6879
Epoch 16/100
55/55 [==============================] - 0s 3ms/step - loss: 0.5239 - accuracy: 0.7807 - val_loss: 0.7258 - val_accuracy: 0.7182
Epoch 17/100
55/55 [==============================] - 0s 1ms/step - loss: 0.5940 - accuracy: 0.7517
Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_27 (Flatten)         (None, 109)               0         
_________________________________________________________________
dense_105 (Dense)            (None, 29)                3190      
_________________________________________________________________
dense_106 (Dense)            (None, 29)                870       
_________________________________________________________________
dense_107 (Dense)            (None, 29)                870       
______________________

55/55 [==============================] - 0s 4ms/step - loss: 1.0410 - accuracy: 0.4838 - val_loss: 0.9640 - val_accuracy: 0.4576
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9058 - accuracy: 0.5128 - val_loss: 0.8966 - val_accuracy: 0.5591
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8631 - accuracy: 0.5839 - val_loss: 0.8684 - val_accuracy: 0.5636
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8444 - accuracy: 0.5856 - val_loss: 0.8534 - val_accuracy: 0.5712
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8334 - accuracy: 0.5861 - val_loss: 0.8434 - val_accuracy: 0.5773
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8259 - accuracy: 0.5952 - val_loss: 0.8358 - val_accuracy: 0.5758
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8200 - accuracy: 0.5958 - val_loss: 0.8302 - val_accuracy: 0.5909
Epoch 8/100
55/55 [======

Epoch 59/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7460 - accuracy: 0.6572 - val_loss: 0.7626 - val_accuracy: 0.6576
Epoch 60/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7453 - accuracy: 0.6578 - val_loss: 0.7620 - val_accuracy: 0.6576
Epoch 61/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7446 - accuracy: 0.6561 - val_loss: 0.7614 - val_accuracy: 0.6561
Epoch 62/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7439 - accuracy: 0.6578 - val_loss: 0.7608 - val_accuracy: 0.6545
Epoch 63/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7432 - accuracy: 0.6561 - val_loss: 0.7602 - val_accuracy: 0.6545
Epoch 64/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7426 - accuracy: 0.6561 - val_loss: 0.7596 - val_accuracy: 0.6545
Epoch 65/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7419 - accuracy: 0.6566 - val_loss: 0.7590 - val_accuracy: 0.6561
Epoch 

Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7594 - accuracy: 0.6449 - val_loss: 0.7519 - val_accuracy: 0.6758
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7585 - accuracy: 0.6438 - val_loss: 0.7513 - val_accuracy: 0.6773
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7574 - accuracy: 0.6489 - val_loss: 0.7507 - val_accuracy: 0.6788
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7564 - accuracy: 0.6472 - val_loss: 0.7503 - val_accuracy: 0.6788
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7554 - accuracy: 0.6472 - val_loss: 0.7496 - val_accuracy: 0.6818
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7544 - accuracy: 0.6477 - val_loss: 0.7491 - val_accuracy: 0.6803
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7535 - accuracy: 0.6472 - val_loss: 0.7486 - val_accuracy: 0.6803
Epoch 

In [23]:
search_1.best_score_

0.7180750296395039

In [40]:
search_1.best_params_

{'clf__activation': 'selu',
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 4,
 'clf__n_neurons': 11,
 'clf__optimizer': 'rmsprop'}

In [41]:
search_1.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7cf42335f8>)])

### FNN With Normalized Input

In [24]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(set_shape_create_model('fnn_np', ncols))

In [25]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', fnn_2)
])

In [26]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [27]:
# %%timeit
# fit model
search_2 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 109)               0         
_________________________________________________________________
dense_128 (Dense)            (None, 3)                 330       
Total params: 330
Trainable params: 330
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 3ms/step - loss: 1.0730 - accuracy: 0.4088 - val_loss: 1.0634 - val_accuracy: 0.3970
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0206 - accuracy: 0.5117 - val_loss: 1.0113 - val_accuracy: 0.5621
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9825 - accuracy: 0.5725 - val_loss: 0.9803 - val_accuracy: 0.5818
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9537 - accuracy: 0.60

55/55 [==============================] - 0s 2ms/step - loss: 0.8982 - accuracy: 0.6157 - val_loss: 0.9124 - val_accuracy: 0.6061
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8825 - accuracy: 0.6236 - val_loss: 0.9024 - val_accuracy: 0.6152
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8677 - accuracy: 0.6282 - val_loss: 0.8904 - val_accuracy: 0.6136
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8567 - accuracy: 0.6259 - val_loss: 0.8805 - val_accuracy: 0.6136
Epoch 10/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8475 - accuracy: 0.6316 - val_loss: 0.8750 - val_accuracy: 0.6121
Epoch 11/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8402 - accuracy: 0.6441 - val_loss: 0.8626 - val_accuracy: 0.6242
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8314 - accuracy: 0.6441 - val_loss: 0.8592 - val_accuracy: 0.6258
Epoch 13/100
55/55 [==

Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7266 - accuracy: 0.6941 - val_loss: 0.7507 - val_accuracy: 0.6894
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7264 - accuracy: 0.6885 - val_loss: 0.7415 - val_accuracy: 0.7076
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7270 - accuracy: 0.6936 - val_loss: 0.7412 - val_accuracy: 0.6985
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7245 - accuracy: 0.6896 - val_loss: 0.7461 - val_accuracy: 0.6985
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7235 - accuracy: 0.6930 - val_loss: 0.7472 - val_accuracy: 0.6909
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7235 - accuracy: 0.6924 - val_loss: 0.7466 - val_accuracy: 0.7015
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7216 - accuracy: 0.6976 - val_loss: 0.7402 - val_accuracy: 0.6985
Epoch 

Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7422 - accuracy: 0.6841 - val_loss: 0.7420 - val_accuracy: 0.6833
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7422 - accuracy: 0.6875 - val_loss: 0.7469 - val_accuracy: 0.6864
Epoch 76/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7418 - accuracy: 0.6869 - val_loss: 0.7390 - val_accuracy: 0.6864
Epoch 77/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7415 - accuracy: 0.6858 - val_loss: 0.7463 - val_accuracy: 0.6788
Epoch 78/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7427 - accuracy: 0.6835 - val_loss: 0.7380 - val_accuracy: 0.6848
Epoch 79/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7395 - accuracy: 0.6869 - val_loss: 0.7426 - val_accuracy: 0.6818
Epoch 80/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7407 - accuracy: 0.6920 - val_loss: 0.7387 - val_accuracy: 0.7000
Epoch 

Epoch 82/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7809 - accuracy: 0.6725 - val_loss: 0.7894 - val_accuracy: 0.6515
Epoch 83/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7803 - accuracy: 0.6754 - val_loss: 0.7888 - val_accuracy: 0.6515
Epoch 84/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7797 - accuracy: 0.6748 - val_loss: 0.7881 - val_accuracy: 0.6485
Epoch 85/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7791 - accuracy: 0.6754 - val_loss: 0.7875 - val_accuracy: 0.6530
Epoch 86/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7786 - accuracy: 0.6754 - val_loss: 0.7870 - val_accuracy: 0.6530
Epoch 87/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7780 - accuracy: 0.6771 - val_loss: 0.7865 - val_accuracy: 0.6500
Epoch 88/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7774 - accuracy: 0.6742 - val_loss: 0.7860 - val_accuracy: 0.6545
Epoch 

...

55/55 [==============================] - 0s 4ms/step - loss: 1.0975 - accuracy: 0.3502 - val_loss: 1.0967 - val_accuracy: 0.3152
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0912 - accuracy: 0.3701 - val_loss: 1.0879 - val_accuracy: 0.3864
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0759 - accuracy: 0.4531 - val_loss: 1.0724 - val_accuracy: 0.4015
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0515 - accuracy: 0.4861 - val_loss: 1.0478 - val_accuracy: 0.4621
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0246 - accuracy: 0.5151 - val_loss: 1.0232 - val_accuracy: 0.5000
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9995 - accuracy: 0.5247 - val_loss: 0.9977 - val_accuracy: 0.5258
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9782 - accuracy: 0.5316 - val_loss: 0.9789 - val_accuracy: 0.5242
Epoch 8/100
55/55 [======

Epoch 59/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8506 - accuracy: 0.5975 - val_loss: 0.8501 - val_accuracy: 0.6076
Epoch 60/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8500 - accuracy: 0.5992 - val_loss: 0.8486 - val_accuracy: 0.6061
Epoch 61/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8492 - accuracy: 0.5930 - val_loss: 0.8477 - val_accuracy: 0.6061
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8486 - accuracy: 0.5964 - val_loss: 0.8468 - val_accuracy: 0.6045
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8478 - accuracy: 0.5981 - val_loss: 0.8463 - val_accuracy: 0.6091
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8472 - accuracy: 0.5986 - val_loss: 0.8448 - val_accuracy: 0.6015
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8465 - accuracy: 0.5992 - val_loss: 0.8441 - val_accuracy: 0.6030
Epoch 

Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8082 - accuracy: 0.6327 - val_loss: 0.8338 - val_accuracy: 0.6197
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8081 - accuracy: 0.6322 - val_loss: 0.8334 - val_accuracy: 0.6212
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8077 - accuracy: 0.6339 - val_loss: 0.8361 - val_accuracy: 0.6136
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8074 - accuracy: 0.6339 - val_loss: 0.8331 - val_accuracy: 0.6197
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8068 - accuracy: 0.6367 - val_loss: 0.8321 - val_accuracy: 0.6197
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8065 - accuracy: 0.6345 - val_loss: 0.8325 - val_accuracy: 0.6136
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8060 - accuracy: 0.6362 - val_loss: 0.8325 - val_accuracy: 0.6152
Epoch 

Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8403 - accuracy: 0.6040 - val_loss: 0.8348 - val_accuracy: 0.6167
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8397 - accuracy: 0.6034 - val_loss: 0.8342 - val_accuracy: 0.6167
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8397 - accuracy: 0.6045 - val_loss: 0.8336 - val_accuracy: 0.6167
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8387 - accuracy: 0.6028 - val_loss: 0.8329 - val_accuracy: 0.6167
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8380 - accuracy: 0.6051 - val_loss: 0.8329 - val_accuracy: 0.6227
Epoch 76/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8377 - accuracy: 0.6000 - val_loss: 0.8321 - val_accuracy: 0.6227
Epoch 77/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8371 - accuracy: 0.6051 - val_loss: 0.8315 - val_accuracy: 0.6212
Epoch 

Epoch 1/100
55/55 [==============================] - 0s 4ms/step - loss: 0.9742 - accuracy: 0.5205 - val_loss: 0.9191 - val_accuracy: 0.5515
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8435 - accuracy: 0.6233 - val_loss: 0.8195 - val_accuracy: 0.6288
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8180 - accuracy: 0.6313 - val_loss: 0.7809 - val_accuracy: 0.6500
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7898 - accuracy: 0.6619 - val_loss: 0.7835 - val_accuracy: 0.6682
Epoch 5/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7780 - accuracy: 0.6659 - val_loss: 0.8005 - val_accuracy: 0.6606
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7682 - accuracy: 0.6642 - val_loss: 0.7622 - val_accuracy: 0.6682
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7597 - accuracy: 0.6659 - val_loss: 0.7456 - val_accuracy: 0.6682
Epoch 8/100
5

In [28]:
search_2.best_score_

0.7074649559814581

In [42]:
search_2.best_params_

{'clf__activation': 'elu',
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 3,
 'clf__n_neurons': 77,
 'clf__optimizer': 'adam'}

In [43]:
search_2.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7cf411e9e8>)])

### CNN With Standardized Input

In [29]:
param_dist = {
    'clf__n_hidden': [1, 2, 3],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
#     'clf__kernel_size': [3,4,5],
    'clf__dropout':[.1, .15, .2, .25]
}

In [30]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn', ncols))

In [31]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', cnn_1)
])

In [32]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [33]:
# %%timeit
# fit model
search_3 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 107, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 53, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 64)            6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 23, 96)            18528     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 96)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 9, 128)            36992   

Epoch 23/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6697 - accuracy: 0.7277 - val_loss: 0.7348 - val_accuracy: 0.6879
Epoch 24/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6683 - accuracy: 0.7265 - val_loss: 0.7292 - val_accuracy: 0.6970
Epoch 25/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6585 - accuracy: 0.7231 - val_loss: 0.7275 - val_accuracy: 0.6864
Epoch 26/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6580 - accuracy: 0.7288 - val_loss: 0.7283 - val_accuracy: 0.6924
Epoch 27/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6499 - accuracy: 0.7294 - val_loss: 0.7271 - val_accuracy: 0.6939
Epoch 28/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6474 - accuracy: 0.7288 - val_loss: 0.7227 - val_accuracy: 0.6909
Epoch 29/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6398 - accuracy: 0.7356 - val_loss: 0.7200 - val_accuracy: 0.7015

Epoch 5/100
55/55 [==============================] - 1s 9ms/step - loss: 0.8224 - accuracy: 0.6335 - val_loss: 0.8157 - val_accuracy: 0.6288
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.8033 - accuracy: 0.6369 - val_loss: 0.8069 - val_accuracy: 0.6197
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7910 - accuracy: 0.6557 - val_loss: 0.7958 - val_accuracy: 0.6288
Epoch 8/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7788 - accuracy: 0.6602 - val_loss: 0.7879 - val_accuracy: 0.6303
Epoch 9/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7692 - accuracy: 0.6670 - val_loss: 0.7795 - val_accuracy: 0.6439
Epoch 10/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7633 - accuracy: 0.6710 - val_loss: 0.7734 - val_accuracy: 0.6470
Epoch 11/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7498 - accuracy: 0.6750 - val_loss: 0.7713 - val_accuracy: 0.6485
Epoch

Epoch 1/100
55/55 [==============================] - 1s 13ms/step - loss: 0.8862 - accuracy: 0.6077 - val_loss: 0.7917 - val_accuracy: 0.6455
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7612 - accuracy: 0.6885 - val_loss: 0.7433 - val_accuracy: 0.6742
Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6669 - accuracy: 0.7334 - val_loss: 0.7674 - val_accuracy: 0.6864
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5971 - accuracy: 0.7561 - val_loss: 0.7389 - val_accuracy: 0.6833
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 0.4913 - accuracy: 0.8039 - val_loss: 0.8378 - val_accuracy: 0.6879
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.3917 - accuracy: 0.8437 - val_loss: 0.9073 - val_accuracy: 0.6591
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.3171 - accuracy: 0.8880 - val_loss: 0.9100 - val_accuracy: 0.6955
Epoch 

Epoch 1/100
55/55 [==============================] - 1s 13ms/step - loss: 0.8862 - accuracy: 0.5926 - val_loss: 0.7808 - val_accuracy: 0.6500
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7729 - accuracy: 0.6756 - val_loss: 0.7467 - val_accuracy: 0.6803
Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6825 - accuracy: 0.7136 - val_loss: 0.7427 - val_accuracy: 0.6864
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5839 - accuracy: 0.7710 - val_loss: 0.7650 - val_accuracy: 0.6939
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 0.4927 - accuracy: 0.8142 - val_loss: 0.8480 - val_accuracy: 0.6348
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.4054 - accuracy: 0.8460 - val_loss: 0.8287 - val_accuracy: 0.7045
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.3357 - accuracy: 0.8795 - val_loss: 0.9092 - val_accuracy: 0.6864
Epoch 

...

Epoch 1/100
55/55 [==============================] - 1s 9ms/step - loss: 0.9099 - accuracy: 0.5767 - val_loss: 0.8311 - val_accuracy: 0.6424
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8251 - accuracy: 0.6426 - val_loss: 0.8073 - val_accuracy: 0.6409
Epoch 3/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7927 - accuracy: 0.6608 - val_loss: 0.7886 - val_accuracy: 0.6530
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7749 - accuracy: 0.6659 - val_loss: 0.7778 - val_accuracy: 0.6500
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7565 - accuracy: 0.6727 - val_loss: 0.7626 - val_accuracy: 0.6712
Epoch 6/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7440 - accuracy: 0.6852 - val_loss: 0.7640 - val_accuracy: 0.6545
Epoch 7/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7357 - accuracy: 0.6852 - val_loss: 0.7472 - val_accuracy: 0.6773
Epoch 8/100
5

Epoch 59/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5520 - accuracy: 0.7733 - val_loss: 0.6762 - val_accuracy: 0.7182
Epoch 60/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5517 - accuracy: 0.7727 - val_loss: 0.6740 - val_accuracy: 0.7242
Epoch 61/100
55/55 [==============================] - 0s 8ms/step - loss: 0.5465 - accuracy: 0.7778 - val_loss: 0.6777 - val_accuracy: 0.7212
Epoch 62/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5463 - accuracy: 0.7727 - val_loss: 0.6730 - val_accuracy: 0.7182
Epoch 63/100
55/55 [==============================] - 0s 8ms/step - loss: 0.5458 - accuracy: 0.7847 - val_loss: 0.6734 - val_accuracy: 0.7182
Epoch 64/100
55/55 [==============================] - 0s 8ms/step - loss: 0.5440 - accuracy: 0.7824 - val_loss: 0.6733 - val_accuracy: 0.7227
Epoch 65/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5398 - accuracy: 0.7852 - val_loss: 0.6743 - val_accuracy: 0.7182
Epoch 

55/55 [==============================] - 1s 11ms/step - loss: 0.9409 - accuracy: 0.5608 - val_loss: 0.8346 - val_accuracy: 0.6076
Epoch 2/100
55/55 [==============================] - 0s 7ms/step - loss: 0.8121 - accuracy: 0.6409 - val_loss: 0.7753 - val_accuracy: 0.6455
Epoch 3/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7176 - accuracy: 0.6926 - val_loss: 0.7591 - val_accuracy: 0.6818
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6365 - accuracy: 0.7364 - val_loss: 0.7955 - val_accuracy: 0.6667
Epoch 5/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5540 - accuracy: 0.7812 - val_loss: 0.8309 - val_accuracy: 0.6591
Epoch 6/100
55/55 [==============================] - 0s 7ms/step - loss: 0.4772 - accuracy: 0.8108 - val_loss: 0.8278 - val_accuracy: 0.6545
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.3804 - accuracy: 0.8500 - val_loss: 0.9479 - val_accuracy: 0.6439
Epoch 8/100
55/55 [=====

Epoch 35/100
83/83 [==============================] - 1s 8ms/step - loss: 0.6026 - accuracy: 0.7537 - val_loss: 0.6745 - val_accuracy: 0.7242
Epoch 36/100
83/83 [==============================] - 1s 7ms/step - loss: 0.6014 - accuracy: 0.7495 - val_loss: 0.6712 - val_accuracy: 0.7227
Epoch 37/100
83/83 [==============================] - 1s 8ms/step - loss: 0.5953 - accuracy: 0.7582 - val_loss: 0.6692 - val_accuracy: 0.7227
Epoch 38/100
83/83 [==============================] - 1s 7ms/step - loss: 0.5929 - accuracy: 0.7579 - val_loss: 0.6710 - val_accuracy: 0.7121
Epoch 39/100
83/83 [==============================] - 1s 7ms/step - loss: 0.5873 - accuracy: 0.7617 - val_loss: 0.6695 - val_accuracy: 0.7197
Epoch 40/100
83/83 [==============================] - 1s 7ms/step - loss: 0.5847 - accuracy: 0.7613 - val_loss: 0.6700 - val_accuracy: 0.7258
Epoch 41/100
83/83 [==============================] - 1s 8ms/step - loss: 0.5898 - accuracy: 0.7628 - val_loss: 0.6707 - val_accuracy: 0.7182
Epoch 

Epoch 93/100
83/83 [==============================] - 1s 8ms/step - loss: 0.5065 - accuracy: 0.7958 - val_loss: 0.6540 - val_accuracy: 0.7227
Epoch 94/100
83/83 [==============================] - 1s 9ms/step - loss: 0.5126 - accuracy: 0.7965 - val_loss: 0.6522 - val_accuracy: 0.7212
Epoch 95/100
83/83 [==============================] - 1s 9ms/step - loss: 0.5080 - accuracy: 0.7980 - val_loss: 0.6525 - val_accuracy: 0.7273
Epoch 96/100
83/83 [==============================] - 1s 9ms/step - loss: 0.5074 - accuracy: 0.8022 - val_loss: 0.6530 - val_accuracy: 0.7258
Epoch 97/100
83/83 [==============================] - 1s 9ms/step - loss: 0.5068 - accuracy: 0.8007 - val_loss: 0.6525 - val_accuracy: 0.7242
Epoch 98/100
83/83 [==============================] - 1s 9ms/step - loss: 0.5018 - accuracy: 0.7984 - val_loss: 0.6513 - val_accuracy: 0.7197
Epoch 99/100
83/83 [==============================] - 1s 9ms/step - loss: 0.5033 - accuracy: 0.8022 - val_loss: 0.6535 - val_accuracy: 0.7242
Epoch 

In [34]:
search_3.best_score_

0.7139067606701911

In [46]:
search_3.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.15,
 'clf__filters': 64,
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 1,
 'clf__n_neurons': 77,
 'clf__optimizer': 'adagrad'}

In [47]:
search_3.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7cf44371d0>)])

### CNN With Normalized Input

In [35]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn_np', ncols))

In [36]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', cnn_1)
])

In [37]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [38]:
# %%timeit
# fit model
search_4 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_89 (Conv1D)           (None, 107, 32)           128       
_________________________________________________________________
max_pooling1d_89 (MaxPooling (None, 53, 32)            0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 51, 64)            6208      
_________________________________________________________________
max_pooling1d_90 (MaxPooling (None, 25, 64)            0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 23, 96)            18528     
_________________________________________________________________
max_pooling1d_91 (MaxPooling (None, 11, 96)            0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 9, 128)            36992   

55/55 [==============================] - 1s 10ms/step - loss: 0.6367 - accuracy: 0.7425 - val_loss: 0.6993 - val_accuracy: 0.6970
Epoch 99/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6365 - accuracy: 0.7470 - val_loss: 0.6985 - val_accuracy: 0.6924
Epoch 100/100
55/55 [==============================] - 0s 4ms/step - loss: 0.6221 - accuracy: 0.7470
Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_93 (Conv1D)           (None, 107, 32)           128       
_________________________________________________________________
max_pooling1d_93 (MaxPooling (None, 53, 32)            0         
_________________________________________________________________
conv1d_94 (Conv1D)           (None, 51, 64)            6208      
_________________________________________________________________
max_pooling1d_94 (MaxPooling (None, 25, 64)            0         
___________________

Epoch 37/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7279 - accuracy: 0.6725 - val_loss: 0.7449 - val_accuracy: 0.6667
Epoch 38/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7175 - accuracy: 0.6850 - val_loss: 0.7432 - val_accuracy: 0.6697
Epoch 39/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7205 - accuracy: 0.6890 - val_loss: 0.7451 - val_accuracy: 0.6545
Epoch 40/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7127 - accuracy: 0.6839 - val_loss: 0.7449 - val_accuracy: 0.6682
Epoch 41/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7157 - accuracy: 0.6782 - val_loss: 0.7395 - val_accuracy: 0.6697
Epoch 42/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7101 - accuracy: 0.6953 - val_loss: 0.7419 - val_accuracy: 0.6697
Epoch 43/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7098 - accuracy: 0.6879 - val_loss: 0.7393 - val_accuracy: 0.6697

Epoch 33/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7365 - accuracy: 0.6790 - val_loss: 0.7417 - val_accuracy: 0.6636
Epoch 34/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7354 - accuracy: 0.6818 - val_loss: 0.7484 - val_accuracy: 0.6576
Epoch 35/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7335 - accuracy: 0.6875 - val_loss: 0.7412 - val_accuracy: 0.6667
Epoch 36/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7292 - accuracy: 0.6830 - val_loss: 0.7406 - val_accuracy: 0.6636
Epoch 37/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7317 - accuracy: 0.6858 - val_loss: 0.7373 - val_accuracy: 0.6667
Epoch 38/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7228 - accuracy: 0.6875 - val_loss: 0.7382 - val_accuracy: 0.6667
Epoch 39/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7256 - accuracy: 0.6864 - val_loss: 0.7454 - val_accuracy: 0.6576

Epoch 1/100
55/55 [==============================] - 1s 14ms/step - loss: 0.9732 - accuracy: 0.4866 - val_loss: 0.8229 - val_accuracy: 0.6273
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7855 - accuracy: 0.6561 - val_loss: 0.8369 - val_accuracy: 0.6394
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7401 - accuracy: 0.6771 - val_loss: 0.7172 - val_accuracy: 0.6970
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6850 - accuracy: 0.7084 - val_loss: 0.7003 - val_accuracy: 0.6818
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6337 - accuracy: 0.7300 - val_loss: 0.7033 - val_accuracy: 0.7167
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6043 - accuracy: 0.7470 - val_loss: 0.7361 - val_accuracy: 0.6530
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5590 - accuracy: 0.7720 - val_loss: 0.6733 - val_accuracy: 0.7121
Epoch 

...

Epoch 16/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8232 - accuracy: 0.6441 - val_loss: 0.8380 - val_accuracy: 0.6288
Epoch 17/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8204 - accuracy: 0.6390 - val_loss: 0.8317 - val_accuracy: 0.6303
Epoch 18/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8175 - accuracy: 0.6435 - val_loss: 0.8329 - val_accuracy: 0.6318
Epoch 19/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8149 - accuracy: 0.6515 - val_loss: 0.8266 - val_accuracy: 0.6273
Epoch 20/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8053 - accuracy: 0.6538 - val_loss: 0.8252 - val_accuracy: 0.6333
Epoch 21/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8077 - accuracy: 0.6595 - val_loss: 0.8207 - val_accuracy: 0.6424
Epoch 22/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8057 - accuracy: 0.6572 - val_loss: 0.8145 - val_accuracy: 0.6364
Epoch 

Epoch 74/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7412 - accuracy: 0.6936 - val_loss: 0.7518 - val_accuracy: 0.6727
Epoch 75/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7345 - accuracy: 0.6981 - val_loss: 0.7552 - val_accuracy: 0.6652
Epoch 76/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7344 - accuracy: 0.6885 - val_loss: 0.7505 - val_accuracy: 0.6727
Epoch 77/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7379 - accuracy: 0.6902 - val_loss: 0.7514 - val_accuracy: 0.6682
Epoch 78/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7304 - accuracy: 0.6993 - val_loss: 0.7488 - val_accuracy: 0.6788
Epoch 79/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7284 - accuracy: 0.6976 - val_loss: 0.7493 - val_accuracy: 0.6712
Epoch 80/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7315 - accuracy: 0.7004 - val_loss: 0.7480 - val_accuracy: 0.6742
Epoch 

Epoch 78/100
55/55 [==============================] - 1s 9ms/step - loss: 0.7251 - accuracy: 0.6879 - val_loss: 0.7619 - val_accuracy: 0.6652
Epoch 79/100
55/55 [==============================] - 0s 9ms/step - loss: 0.7206 - accuracy: 0.6919 - val_loss: 0.7615 - val_accuracy: 0.6591
Epoch 80/100
55/55 [==============================] - 0s 9ms/step - loss: 0.7212 - accuracy: 0.6947 - val_loss: 0.7615 - val_accuracy: 0.6621
Epoch 81/100
55/55 [==============================] - 1s 9ms/step - loss: 0.7211 - accuracy: 0.6924 - val_loss: 0.7558 - val_accuracy: 0.6652
Epoch 82/100
55/55 [==============================] - 0s 9ms/step - loss: 0.7206 - accuracy: 0.6993 - val_loss: 0.7540 - val_accuracy: 0.6697
Epoch 83/100
55/55 [==============================] - 1s 9ms/step - loss: 0.7211 - accuracy: 0.6953 - val_loss: 0.7571 - val_accuracy: 0.6742
Epoch 84/100
55/55 [==============================] - 0s 9ms/step - loss: 0.7200 - accuracy: 0.6907 - val_loss: 0.7570 - val_accuracy: 0.6727
Epoch 

Epoch 1/100
55/55 [==============================] - 1s 11ms/step - loss: 0.9841 - accuracy: 0.4929 - val_loss: 0.8685 - val_accuracy: 0.6015
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8076 - accuracy: 0.6367 - val_loss: 0.7501 - val_accuracy: 0.6712
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7204 - accuracy: 0.6885 - val_loss: 0.7306 - val_accuracy: 0.6848
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6805 - accuracy: 0.7067 - val_loss: 0.6988 - val_accuracy: 0.7061
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6388 - accuracy: 0.7288 - val_loss: 0.7235 - val_accuracy: 0.6848
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6030 - accuracy: 0.7499 - val_loss: 0.6915 - val_accuracy: 0.6939
Epoch 7/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5634 - accuracy: 0.7715 - val_loss: 0.7078 - val_accuracy: 0.6985
Epoch 8/100


Epoch 1/100
55/55 [==============================] - 1s 10ms/step - loss: 0.9916 - accuracy: 0.4852 - val_loss: 0.8278 - val_accuracy: 0.6152
Epoch 2/100
55/55 [==============================] - 0s 7ms/step - loss: 0.8133 - accuracy: 0.6307 - val_loss: 0.7643 - val_accuracy: 0.6530
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7341 - accuracy: 0.6830 - val_loss: 0.8409 - val_accuracy: 0.6091
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6966 - accuracy: 0.6960 - val_loss: 0.7144 - val_accuracy: 0.6879
Epoch 5/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6589 - accuracy: 0.7080 - val_loss: 0.7200 - val_accuracy: 0.6788
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6129 - accuracy: 0.7284 - val_loss: 0.7829 - val_accuracy: 0.6591
Epoch 7/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5728 - accuracy: 0.7437 - val_loss: 0.7278 - val_accuracy: 0.6773
Epoch 8/100


In [39]:
search_4.best_score_

0.7180750323498326

In [48]:
search_4.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.1,
 'clf__filters': 64,
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 1,
 'clf__n_neurons': 77,
 'clf__optimizer': 'adam'}

In [49]:
search_4.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7cf415c198>)])